<a href="https://colab.research.google.com/github/dan-a-iancu/OIT248/blob/main/Whole_Wallet/Whole_Wallet_Beyond_Gurobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook shows you how to implement and solve Q1 from the Whole Wallet minicase using freely available modeling software and solvers.**

# Basic Setup

Import necessary packages si define some helper functions.

In [1]:
# --- Colab setup ---

#---- Install Gurobi ---
!pip install gurobipy
from gurobipy import *

# --- Install PuLP (this comes with CBC - free LP solver) ---
!pip -q install pulp

# --- Install Pyomo + CBC ---
!pip -q install pyomo
!apt -y -q install coinor-cbc   # install the cbc solver from CoinOR

# --- Google OR-Tools ---
!pip -q install ortools

# --- Helper functions (getNeighbors, etc) ---
import numpy as np, pandas as pd
import time
from itertools import product

def in_bounds(r,c): return (0 <= r < len(rows)) and (0 <= c < len(columns))
def getNeighbors(r,c):
    # 8-neighborhood (incl. diagonals)
    delta = [-1,0,1]
    out = []
    for dr in delta:
        for dc in delta:
            if dr==0 and dc==0: continue
            rr, cc = r+dr, c+dc
            if in_bounds(rr,cc): out.append((rr,cc))
    return out

### ------- A function to generate a random problem instance -----------
def generate_random_instance(nrows, ncols):
      # Set the random number seed
      np.random.seed(0)

      # Adjust the size of the grid (i.e., number of districts)
      rows = range(nrows)
      columns = range(ncols)

      # Example data
      MarketSizes = pd.DataFrame(
          np.random.randint(8_000, 20_000, size=(len(rows), len(columns))),
          index=rows, columns=columns
      )
      Incomes = pd.DataFrame(
          np.random.randint(400, 1200, size=(len(rows), len(columns))),
          index=rows, columns=columns
      )

      return MarketSizes, Incomes

Reading package lists...
Building dependency tree...
Reading state information...
coinor-cbc is already the newest version (2.10.7+ds1-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


# The data from the case

In [2]:
attractInDistrict = 0.65
attractNeighboringDistrict = 0.25
costPerStore = 1_900_000.0

In [3]:
### A boolean to determine whether to solve a test instance for each model below
solve_an_instance = False

# Construct the model in Gurobi
Define a function that creates a model like in **Q1** using Gurobi

In [4]:
### Define a function that creates a model like in Q1 in Gurobi
def create_model_like_Q1_gurobi():
    model = Model("Whole Wallet Supermarket Operations")
    X = model.addVars(rows,columns, vtype=GRB.BINARY, name="Open")

    ### OPTION 1 for calculating revenues
    hypotheticalRevenues = {}  # create an empty dictionary to store these hypothetical revenues
    for r in rows:
        for c in columns:
            # add the revenues made from customers from district (r,c)
            hypotheticalRevenues[r,c] = (attractInDistrict * MarketSizes.loc[r,c] * Incomes.loc[r,c])

            # add revenues made from customers coming from neighboring districts
            hypotheticalRevenues[r,c] += sum( attractNeighboringDistrict*MarketSizes.loc[i,j]*Incomes.loc[i,j] for (i,j) in getNeighbors(r,c) )

    ## Now calculate revenues
    revenues = quicksum( X[r,c]*hypotheticalRevenues[r,c] for r in rows for c in columns )
    costs = costPerStore * quicksum ( X[r,c] for r in rows for c in columns )
    model.setObjective(revenues-costs, GRB.MAXIMIZE)

    ### OPTION 1 for the adjacency constraints
    for r in rows:
        for c in columns:
            neigh = getNeighbors(r,c)  # get the list of neighbors
            for (i,j) in neigh:
                model.addConstr( X[r,c] + X[i,j] <= 1 )

    return model, X

### Solve an instance
if solve_an_instance:

    # generate a random instance with given size (in terms of number of rows/columns)
    MarketSizes, Incomes = generate_random_instance(nrows=10, ncols=10)
    rows = MarketSizes.index
    columns = MarketSizes.columns

    model_gb, X = create_model_like_Q1_gurobi()

    # Solve with Gurobi
    model_gb.setParam('OutputFlag', False)
    start = time.perf_counter()
    model_gb.optimize()
    end = time.perf_counter()
    model_gb.optimize()

    print(f"Optimal profit: {model_gb.objVal:,.2f}")
    chosen = [(r,c) for r in rows for c in columns if X[r,c].X==1]

    print("Open stores at: ", chosen)

# Construct the model in PuLP and solve it with CBC
Define a function that creates a model like in **Q1** using the PuLP modeling language and solves it using CBC

In [5]:
# --- PuLP model matching our implementation ---
import pulp as pl

def create_model_like_Q1_pulp():
    model = pl.LpProblem("Whole_Wallet_Supermarket_Operations", pl.LpMaximize)
    X = pl.LpVariable.dicts("Open",
        ((r,c) for r in rows for c in columns), lowBound=0, upBound=1, cat=pl.LpBinary
    )

    # OPTION 1: additive hypothetical revenues (your approach)
    hypotheticalRevenues = {}
    for r in rows:
        for c in columns:
            own = attractInDistrict * MarketSizes.loc[r,c] * Incomes.loc[r,c]
            neigh = sum(
                attractNeighboringDistrict * MarketSizes.loc[i,j] * Incomes.loc[i,j]
                for (i,j) in getNeighbors(r,c)
            )
            hypotheticalRevenues[(r,c)] = own + neigh

    # Objective: sum(X * hypotheticalRevenues) - cost * sum(X)
    revenue = pl.lpSum(X[(r,c)] * hypotheticalRevenues[(r,c)] for r in rows for c in columns)
    cost = costPerStore * pl.lpSum(X[(r,c)] for r in rows for c in columns)
    model += (revenue - cost)

    # Adjacency constraints (duplicated on purpose for clarity)
    for r in rows:
        for c in columns:
            for (i,j) in getNeighbors(r,c):
                model += (X[(r,c)] + X[(i,j)] <= 1, f"NoAdj_{r}_{c}_{i}_{j}")

    return model, X

if solve_an_instance:
    model, X = create_model_like_Q1_pulp()

    MarketSizes, Incomes = generate_random_instance(nrows=10, ncols=10)
    rows = MarketSizes.index
    columns = MarketSizes.columns

    # Solve with CBC
    _ = model.solve(pl.PULP_CBC_CMD(msg=False))

    print(f"Objective value: {pl.value(model.objective):,.2f}")

    # Show chosen store locations
    chosen = [(r,c) for r in rows for c in columns if pl.value(X[(r,c)]) > 0.5]
    print("Open stores at:", chosen)

# Construct the model in Pyomo and solve using CBC

In [6]:
# --- Pyomo model using your same structure (additive uplift, duplicated adjacencies) ---
import pyomo.environ as pyo

def create_model_like_Q1_pyomo():
    m = pyo.ConcreteModel()

    # Sets
    m.R = pyo.RangeSet(0, len(rows)-1)
    m.C = pyo.RangeSet(0, len(columns)-1)

    # Variables
    m.X = pyo.Var(m.R, m.C, within=pyo.Binary)

    # Precompute hypothetical revenues like in your code
    hyp = {}
    for r in rows:
        for c in columns:
            own = attractInDistrict * MarketSizes.loc[r,c] * Incomes.loc[r,c]
            neigh = sum(
                attractNeighboringDistrict * MarketSizes.loc[i,j] * Incomes.loc[i,j]
                for (i,j) in getNeighbors(r,c)
            )
            hyp[(r,c)] = own + neigh

    # Objective
    def obj_rule(m):
        revenue = sum(m.X[r,c] * hyp[(r,c)] for r in rows for c in columns)
        cost = costPerStore * sum(m.X[r,c] for r in rows for c in columns)
        return revenue - cost
    m.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

    # Adjacency constraints (duplicated)
    m.NoAdj = pyo.ConstraintList()
    for r in rows:
        for c in columns:
            for (i,j) in getNeighbors(r,c):
                m.NoAdj.add(m.X[r,c] + m.X[i,j] <= 1)

    return m

if solve_an_instance:

    MarketSizes, Incomes = generate_random_instance(nrows=10, ncols=10)
    rows = MarketSizes.index
    columns = MarketSizes.columns

    m = create_model_like_Q1_pyomo()
    solver = pyo.SolverFactory('cbc')   # <-- use CBC instead of GLPK
    res = solver.solve(m, tee=False)
    print(res.solver.status, res.solver.termination_condition)

    # Extract results
    open_sites = [(r,c) for r in rows for c in columns if pyo.value(m.X[r,c]) > 0.5]
    obj = pyo.value(m.OBJ)
    print(f"Objective value: {obj:,.2f}")
    print("Open stores at:", open_sites)

# Construct the model in Google OR Tools

In [7]:
# --- OR-Tools CP-SAT model (same structure; integerized objective) ---
from ortools.sat.python import cp_model
import math

def create_model_like_Q1_ortools(scale=1):
    model = cp_model.CpModel()

    # Variables
    X = {}
    for r in rows:
        for c in columns:
            X[(r,c)] = model.NewBoolVar(f"Open_{r}_{c}")

    # Integerized hypothetical revenues
    hyp_int = {}
    for r in rows:
        for c in columns:
            own = attractInDistrict * MarketSizes.loc[r,c] * Incomes.loc[r,c]
            neigh = sum(
                attractNeighboringDistrict * MarketSizes.loc[i,j] * Incomes.loc[i,j]
                for (i,j) in getNeighbors(r,c)
            )
            hyp = own + neigh
            hyp_int[(r,c)] = int(round(scale * hyp))

    cost_int = int(round(scale * costPerStore))

    # Objective: maximize sum hyp_int * X - cost_int * X
    model.Maximize(
        sum(hyp_int[(r,c)] * X[(r,c)] for r in rows for c in columns)
        - cost_int * sum(X[(r,c)] for r in rows for c in columns)
    )

    # Adjacency constraints (duplicated on purpose)
    for r in rows:
        for c in columns:
            for (i,j) in getNeighbors(r,c):
                model.Add(X[(r,c)] + X[(i,j)] <= 1)

    return model, X

if solve_an_instance:

    MarketSizes, Incomes = generate_random_instance(nrows=10, ncols=10)
    rows = MarketSizes.index
    columns = MarketSizes.columns

    model, X = create_model_like_Q1_ortools(scale=1)
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10.0
    status = solver.Solve(model)

    print("Status:", solver.StatusName(status))
    obj_val = solver.ObjectiveValue()
    print("Objective value (scaled):", obj_val)

    chosen = [(r,c) for r in rows for c in columns if solver.Value(X[(r,c)]) == 1]
    print("Open stores at:", chosen)

# Performance benchmarking

In [9]:
##### Set up a problem instance to use
MarketSizes, Incomes = generate_random_instance(nrows=10, ncols=10)
rows = MarketSizes.index
columns = MarketSizes.columns

##### Gurobi -- only works for problems with at most 2000 districts
model_gb, X = create_model_like_Q1_gurobi()
model_gb.setParam('OutputFlag', False)
start = time.perf_counter()   # start timer
model_gb.optimize()
end = time.perf_counter()     # end timer
print(f"Gurobi optimal profit: {model_gb.objVal:,.2f}")
print(f"Gurobi solve time: {end-start:.5} sec")

##### PuLP + CBC
model, X = create_model_like_Q1_pulp()
start = time.perf_counter()
_ = model.solve(pl.PULP_CBC_CMD(msg=False))
end = time.perf_counter()
print(f"\nPuLP + CBC objective value: {pl.value(model.objective):,.2f}")
print(f"PuLP + CBC solver time: {end-start:.5} sec")

##### Google OR Tools
model, X = create_model_like_Q1_ortools(scale=1)
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10.0
start = time.perf_counter()
status = solver.Solve(model)
end = time.perf_counter()

print(f"\nGoogle OR Tools value (scaled): {solver.ObjectiveValue():,.2f}")
print(f"Google OR Tools solver time: {end-start:.5} sec")

Gurobi optimal profit: 616,279,670.15
Gurobi solve time: 0.0079057 sec

PuLP + CBC objective value: 616,279,670.15
PuLP + CBC solver time: 0.072307 sec

Google OR Tools value (scaled): 616,279,672.00
Google OR Tools solver time: 0.028157 sec
